In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("iris/iris.csv", header=None)

In [3]:
def encode_textdata(ds, column):
    ids = {name: id for name, id in zip(list(ds[column].unique()), range(ds.shape[0]))}
    return ids, ds[column].transform(lambda x: ids[x])

In [4]:
ids, dataslice = encode_textdata(dataset, 4)

In [5]:
x = dataset[[0, 1, 2, 3]]

In [6]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.fit_transform(x)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, dataslice, test_size=0.3, random_state=42)

In [8]:
X_train

array([[-4.16009689e-01, -1.51337539e+00, -3.33612130e-02,
        -2.61192967e-01],
       [ 5.53333275e-01, -5.87763531e-01,  7.62758643e-01,
         3.96171883e-01],
       [ 6.74501145e-01,  1.06445364e-01,  9.90221459e-01,
         7.90590793e-01],
       [ 9.16836886e-01, -1.24957601e-01,  3.64698715e-01,
         2.64698913e-01],
       [ 1.64384411e+00,  1.26346019e+00,  1.33141568e+00,
         1.71090158e+00],
       [-1.73673948e-01, -3.56360566e-01,  2.50967307e-01,
         1.33225943e-01],
       [ 2.12851559e+00, -1.24957601e-01,  1.61574420e+00,
         1.18500970e+00],
       [-2.94841818e-01, -1.24957601e-01,  4.21564419e-01,
         3.96171883e-01],
       [-9.00681170e-01,  1.03205722e+00, -1.34127240e+00,
        -1.31297673e+00],
       [ 2.24968346e+00, -5.87763531e-01,  1.67260991e+00,
         1.05353673e+00],
       [-5.25060772e-02, -8.19166497e-01,  1.94101603e-01,
        -2.61192967e-01],
       [-7.79513300e-01,  8.00654259e-01, -1.34127240e+00,
      

In [9]:
sepal_length = ctrl.Antecedent(np.arange(x[:, 0].min(), x[:, 0].max(), 0.1), 'sepal_length')
sepal_width = ctrl.Antecedent(np.arange(x[:, 1].min(), x[:, 1].max(), 0.1), 'sepal_width')
petal_length = ctrl.Antecedent(np.arange(x[:, 2].min(), x[:, 2].max(), 0.1), 'petal_length')
petal_width = ctrl.Antecedent(np.arange(x[:, 3].min(), x[:, 3].max(), 0.1), 'petal_width')
y_class = ctrl.Consequent(np.arange(0, 3, 1), 'y_class')
y_class['Iris-setosa'] = fuzz.trimf(y_class.universe, [0, 0, 0.5])
y_class['Iris-versicolor'] = fuzz.trimf(y_class.universe, [0, 1, 2])
y_class['Iris-virginica'] = fuzz.trimf(y_class.universe, [1.5, 2, 2])

In [10]:
sepal_length_short = fuzz.trimf(sepal_length.universe, [x[:, 0].min(), x[:, 0].min(), x[:, 0].mean()])
sepal_length_medium = fuzz.trimf(sepal_length.universe, [x[:, 0].min(), x[:, 0].mean(), x[:, 0].max()])
sepal_length_long = fuzz.trimf(sepal_length.universe, [x[:, 0].mean(), x[:, 0].max(), x[:, 0].max()])

sepal_width_narrow = fuzz.trapmf(sepal_width.universe, [x[:, 1].min(), x[:, 1].min(), x[:, 1].mean(), x[:, 1].mean()+0.5])
sepal_width_medium = fuzz.trimf(sepal_width.universe, [x[:, 1].min(), x[:, 1].mean(), x[:, 1].max()])
sepal_width_wide = fuzz.trapmf(sepal_width.universe, [x[:, 1].mean(), x[:, 1].mean()+0.5, x[:, 1].max(), x[:, 1].max()])

petal_length_short = fuzz.trimf(petal_length.universe, [x[:, 2].min(), x[:, 2].min(), x[:, 2].mean()])
petal_length_medium = fuzz.trimf(petal_length.universe, [x[:, 2].min(), x[:, 2].mean(), x[:, 2].max()])
petal_length_long = fuzz.trimf(petal_length.universe, [x[:, 2].mean(), x[:, 2].max(), x[:, 2].max()])

petal_width_narrow = fuzz.trapmf(petal_width.universe, [x[:, 3].min(), x[:, 3].min(), x[:, 3].mean(), x[:, 3].mean()+0.5])
petal_width_medium = fuzz.trimf(petal_width.universe, [x[:, 3].min(), x[:, 3].mean(), x[:, 3].max()])
petal_width_wide = fuzz.trapmf(petal_width.universe, [x[:, 3].mean(), x[:, 3].mean()+0.5, x[:, 3].max(), x[:, 3].max()])

In [16]:
rule1 = ctrl.Rule(sepal_length_short & sepal_width_narrow & petal_length_short & petal_width_narrow, iris_class['Iris-setosa'])
rule2 = ctrl.Rule(sepal_length_medium & sepal_width_medium & petal_length_medium & petal_width_medium, iris_class['Iris-versicolor'])
rule3 = ctrl.Rule(sepal_length_long & sepal_width_wide & petal_length_long & petal_width_wide, iris_class['Iris-virginica'])

iris_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
iris_system = ctrl.ControlSystemSimulation(iris_ctrl)

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''